# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-22-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
# Create a Keyspace to store tables
try:
    session.execute('''
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    {'class':'SimpleStrategy','replication_factor':1}
    '''
)
except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
# Set KEYSPACE to the keyspace sparkifydb
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

This query reflects the user's desire to know information about a given song a user played during a session. This query presumes the user knows a particular user session id, as well as session item number. The database built for this query could also be run to utlize all songs in a given session, as the session id will be the first filtering criteria. The table is organized so that the information the user wants to see (artist, song, and song length) is towards the left hand side of the table, where the user will look first.

As only the song-related info is wanted here, I will remove some of the user fields that are unnecessary, such as level and location.

#### Create Insert function 

In [5]:
# create dictionary of header columns and index for my reference when inserting
header = ['artist', 'firstName', 'gender', 'itemInSession', 'lastName',
          'length', 'level', 'location', 'sessionId', 'song', 'userId']
header = dict(enumerate(header))
header

{0: 'artist',
 1: 'firstName',
 2: 'gender',
 3: 'itemInSession',
 4: 'lastName',
 5: 'length',
 6: 'level',
 7: 'location',
 8: 'sessionId',
 9: 'song',
 10: 'userId'}

#### Create Table

In [6]:
## 1.  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS session_item_info "
query = query + '''(session_id int,
                    item_in_session int,
                    user_id int ,
                    artist text,
                    song text,
                    length double,
                    first_name text,
                    last_name text,
                    gender text,
                    level text,
                    location text,
                    PRIMARY KEY (session_id, item_in_session))'''
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert Data

In [9]:
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO session_item_info 
                            (session_id ,
                            item_in_session,
                            user_id,
                            artist,
                            song,
                            length)'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), 
                                int(line[3]), 
                                int(line[10]),
                                line[0],
                                line[9],
                                float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# 1. Give me the artist, song title and song's length during sessionId = 338, and itemInSession = 4
query = 'SELECT artist, song, length FROM session_item_info WHERE session_id = 338 AND item_in_session=4 '
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

This query is similar to the first, in that you are querying to get the name of an artist and song, but there are a few more additions. User info, such as first/last name are needed, as well as the results being sorted by item_in_session (or the order they were played). These additional requirements push the user_id to front as the first partition key, and uses item_in_session as a clustering key for sorting.

As this query wants user information, I will keep all of those related fields in the table, such as location, but remove length.

#### Create Table

In [14]:
## create table that uses the user id and session id as the partition keys,
#   and item in session as the clustering key for sorting
query = "CREATE TABLE IF NOT EXISTS user_session_info "
query = query + '''(user_id int,
                    session_id int ,
                    item_in_session int,
                    first_name text,
                    last_name text,
                    gender text,
                    level text,
                    location text, 
                    artist text,
                    song text, 
                    PRIMARY KEY ((user_id, session_id), item_in_session))'''
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert Data

In [15]:
# insert data into user_session_info
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO user_session_info 
                            (session_id ,item_in_session,
                            user_id, first_name,
                            last_name, gender,
                            level, location,
                            artist,song)'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]), 
                                int(line[10]), line[1],
                                line[4],line[2],
                                line[6],line[7],
                                line[0],line[9]))

#### Execute Query to Verify

In [19]:
## 2. Give me only the following: name of artist, song (sorted by itemInSession) and 
# user (first and last name) for userid = 10, sessionid = 182
query = 'SELECT artist, song, first_name, last_name FROM user_session_info WHERE user_id = 10 AND session_id=182 '
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                    

This song is much different than the first 2, in that we want to filter for 1 criteria (song). Unlike the first 2 queries, the song field does not have natural primary key groupings (like session_id + item_in_session), so we will create one here using song + sessionId. This primary key order will allow us to filter for song and preserve uniqueness.  

This query wants to view user information for a specific song, so I will keep user related fields in, like level and location, and remove others like song length and artist from the table.

#### Create Table

In [29]:
## create table that uses the song name for filtering, as no song_id exists (though could be created)
# as well as session id and item in session for sorting as the partition keys,
query = "CREATE TABLE IF NOT EXISTS user_song_info "
query = query + '''(song text,
                    session_id int,
                    item_in_session int,
                    first_name text,
                    last_name text,
                    user_id int,
                    gender text,
                    level text,
                    location text, 
                    PRIMARY KEY (song, session_id))'''
try:
    session.execute(query)
except Exception as e:
    print(e)           

#### Insert Data

In [30]:
# insert data
with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''INSERT INTO user_song_info 
                            (session_id, item_in_session,
                            user_id, first_name,
                            last_name, gender,
                            level, location,
                            song)'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), 
                                int(line[10]),line[1],
                                line[4],line[2],
                                line[6],line[7],
                                line[9]))

#### Execute Query to Verify

In [31]:
## 3. Give me every user name (first and last) in my music app history who
#   listened to the song 'All Hands Against His Own'
query = "SELECT first_name, last_name FROM user_song_info WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)                

Row(first_name='Sara', last_name='Johnson')
Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')


### Drop the tables before closing out the sessions

In [32]:
queries = ['DROP TABLE IF EXISTS user_song_info',
           'DROP TABLE IF EXISTS user_session_info',
           'DROP TABLE IF EXISTS session_item_info']

for query in queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)     

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()